In [321]:
import numpy as np
from tqdm import tqdm

In [331]:
#check when no hidden layer is passed
#currently only work for single output
class MiniFlow():
     #input - actual input, output - actual output , hidden layer - array of size of hidden layers
     # preprocess_function - it will return train_data,train_output
     # learning_rate = how fast learning rate should be.
        
    def __init__(self,inputs,outputs,hidden_layers,preprocess_function,learning_rate = 0.1):
        
        np.random.seed(0)
        self.inputs,self.outputs = preprocess_function(inputs,outputs)
        self.initialization(inputs,outputs,hidden_layers,learning_rate)
        
    def initialization(self,inputs,outputs,hidden_layers,learning_rate):
        
        self.learning_rate = learning_rate
        self.number_of_hidden_layers = len(hidden_layers)
        self.weights =  dict()
        self.output_size = outputs.shape[1]
        row = inputs.shape[1]
        col = hidden_layers[0]
        
        counter = 0
        for i in range(self.number_of_hidden_layers+1):
            print('row',row,'col',col)
            self.weights.update({i:np.random.rand(row,col)})
            row = col
            if i + 1 >= self.number_of_hidden_layers:
                col = self.output_size
            else:
                col = hidden_layers[counter+1]
            counter = counter+1
            
    def get_weights(self):
        return self.weights                
            
    def train(self,number_of_epoch,batch_size):
        
        #currently we not using epoch and batch size
        print(self.weights)
        for j in tqdm(range(number_of_epoch)):
            batch = np.random.choice(len(self.inputs), size=batch_size)
            X,Y = self.inputs[batch],self.outputs[batch]       
            for i in range(len(X)):
                self.feed_forward_backward(X[i],Y[i],i)
        print(self.weights)                    
        pass
    def feed_forward(self,inputs):
        
        result = inputs
        self.temp_inputs = dict()
        for i in range(self.number_of_hidden_layers+1):
            self.temp_inputs.update({i:np.asmatrix(result)})
            result = np.dot(result,self.weights[i])
        result = self.sigmoid(result)
        return result

    def feed_forward_backward(self,inputs,outputs,counter_value):
        
        prediction = self.feed_forward(inputs)

        #last layer error and delta
        error = self.error_calculation(outputs,prediction)
        error_delta = error * self.sigmoid_derivative(prediction)        
        error_delta = error_delta.reshape(1,1)

        #actual back propogation
        for i in reversed(range(self.number_of_hidden_layers+1)):

            current_weight = self.weights[i]
            temp = (self.temp_inputs[i].T.dot(error_delta)*self.learning_rate)
            
            self.weights[i] = self.weights[i] + temp
            error = np.asmatrix(error_delta.dot(current_weight.T))
            error_delta = error
        
        
    def error_calculation(self,actual,predicted):
        return actual - predicted
        
    def sigmoid(self,data):
        return 1/(1+np.exp(-data))
    
    def sigmoid_derivative(self,data):
        return data * (1- data)
            

In [243]:
test_input = np.array([[2.9,2.4,3.0,4.2],[5.0,6.0,7.0,8.0]],dtype=float)
test_label = np.array([[0],[1]])
test_hidden_layers = [2]

def preprocess_function(inputs,outputs):
    return inputs,outputs

In [244]:
netowrk = MiniFlow(inputs=test_input,outputs=test_label,hidden_layers=test_hidden_layers,
                   preprocess_function=preprocess_function)

row 4 col 2
row 2 col 1


In [245]:
netowrk.get_weights()

{0: array([[ 0.5488135 ,  0.71518937],
        [ 0.60276338,  0.54488318],
        [ 0.4236548 ,  0.64589411],
        [ 0.43758721,  0.891773  ]]), 1: array([[ 0.96366276],
        [ 0.38344152]])}

In [246]:
-0.99991724 * netowrk.sigmoid_derivative(0.99991724)

-8.27463021316008e-05

In [247]:
netowrk.train(1,1)

In [248]:
netowrk.get_weights()

{0: matrix([[ 0.54883663,  0.71519857],
         [ 0.60278251,  0.5448908 ],
         [ 0.42367872,  0.64590363],
         [ 0.4376207 ,  0.89178633]]), 1: matrix([[ 0.96371362],
         [ 0.38351653]])}

In [250]:
print(test_input.shape)
print(test_label.shape)

(2, 4)
(2, 1)


Test MiniFLow Network on data set

In [251]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

In [332]:
raw_data = pd.read_csv('HR_comma_sep.csv')

In [333]:
salary_mapping = {'low':1,'medium':3,'high':5}
raw_data['salary'] = raw_data.salary.replace(salary_mapping)
dummies = pd.get_dummies(raw_data['sales'], prefix='sales', drop_first=False)
raw_data = pd.concat([raw_data, dummies], axis=1)
raw_data = raw_data.drop('sales',axis=1)


In [334]:
columns_to_be_normalized = ['number_project','average_montly_hours','time_spend_company','salary']
scaled_features = {}
for each in columns_to_be_normalized:
    mean, std = raw_data[each].mean(), raw_data[each].std()
    scaled_features[each] = [mean, std]
    raw_data.loc[:, each] = (raw_data[each] - mean)/std

In [335]:
number_of_columns = raw_data.columns.size
number_of_rows = len(raw_data)
test_size = 1000
validation_size = 3000
target_field = ['left']

In [336]:
raw_data = shuffle(raw_data)

In [337]:
test_data = raw_data[:test_size]
validation_data = raw_data[test_size:test_size+validation_size]
train_data = raw_data[test_size+validation_size:len(raw_data)]

In [338]:
train_features,train_target = train_data.drop(target_field,axis=1),train_data[target_field]
validation_features,validation_target = validation_data.drop(target_field,axis=1),validation_data[target_field]
test_features,test_target = test_data.drop(target_field,axis=1),test_data[target_field]

In [339]:
def process_hr_data(inputs,outputs):
    return inputs,outputs

In [340]:
def MSE(y, Y):
    return np.mean((y-Y)**2)

In [281]:
train_features.shape

(10999, 18)

In [282]:
test_features.shape

(1000, 18)

In [347]:
neuralNetwork =  MiniFlow(inputs=train_features.values,outputs=train_target.values,hidden_layers=[9,5],learning_rate=0.1,preprocess_function=process_hr_data)

row 18 col 9
row 9 col 5
row 5 col 1


In [ ]:
neuralNetwork.train(25000,128)

  0%|          | 7/25000 [00:00<06:19, 65.94it/s]

{0: array([[ 0.5488135 ,  0.71518937,  0.60276338,  0.54488318,  0.4236548 ,
         0.64589411,  0.43758721,  0.891773  ,  0.96366276],
       [ 0.38344152,  0.79172504,  0.52889492,  0.56804456,  0.92559664,
         0.07103606,  0.0871293 ,  0.0202184 ,  0.83261985],
       [ 0.77815675,  0.87001215,  0.97861834,  0.79915856,  0.46147936,
         0.78052918,  0.11827443,  0.63992102,  0.14335329],
       [ 0.94466892,  0.52184832,  0.41466194,  0.26455561,  0.77423369,
         0.45615033,  0.56843395,  0.0187898 ,  0.6176355 ],
       [ 0.61209572,  0.616934  ,  0.94374808,  0.6818203 ,  0.3595079 ,
         0.43703195,  0.6976312 ,  0.06022547,  0.66676672],
       [ 0.67063787,  0.21038256,  0.1289263 ,  0.31542835,  0.36371077,
         0.57019677,  0.43860151,  0.98837384,  0.10204481],
       [ 0.20887676,  0.16130952,  0.65310833,  0.2532916 ,  0.46631077,
         0.24442559,  0.15896958,  0.11037514,  0.65632959],
       [ 0.13818295,  0.19658236,  0.36872517,  0.82099323

 56%|█████▋    | 14085/25000 [02:53<02:11, 82.89it/s]

In [301]:
a = np.random.choice(len(test_features), size=3)
print(a)

[118 545 151]


In [303]:
test_features.values[a]

array([[ 0.69      ,  0.93      , -0.65151592,  0.63972128, -0.34122379,
         0.        ,  0.        ,  0.63607114,  1.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ],
       [ 0.29      ,  0.77      , -0.65151592, -0.9821244 , -0.34122379,
         1.        ,  0.        , -0.93333683,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ],
       [ 0.64      ,  0.9       ,  1.78237878, -1.16232948,  1.02851142,
         0.        ,  0.        ,  2.20547911,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ]])

In [346]:
correct = 0 
for i in range(len(test_features.values)):
    r = neuralNetwork.feed_forward(test_features.values[i])
    temp = 0
    if r > 0.50:
        temp = 1 
    else :
        temp = 0   
    if(test_target.values[i][0] == temp):
        correct = correct+1    
        
print(correct/len(test_data)*100)        

84.0
